# Vineyards navigation with semantic segmentation

Using semantic segmentation for a proportional controller along the vineyard rows.




- This notebook contains scripts to create, train, and test a deep learning network to perform fast semantic segmentation on platform with mobile CPUs and low memory capabilities

- The implemented architecture is a MobileNetV3 with a customized LR-ASSP

# 0 Initialization

In [1]:
#Required libraries

import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from tqdm.notebook import tqdm as tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

2022-10-21 23:58:17.777320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
#select the working GPU

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)


2022-10-21 23:58:18.699719: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-21 23:58:18.700638: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-21 23:58:19.217262: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:60:00.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.41GiB deviceMemoryBandwidth: 1.41TiB/s
2022-10-21 23:58:19.219300: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:61:00.0 name: NVIDIA A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.41GiB deviceMemoryBandwidth: 1.41TiB/s
2022-10-21 23:58:19.221255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:da:00.0 name: NVID

# 1.0 Import the dataset

In [3]:
PATH_DIR = 'datasets_segmentation'
training_dataset_dir = os.path.join(PATH_DIR, 'train')
test_dataset_dir = os.path.join(PATH_DIR, 'test')

In [4]:
train_img_list = glob.glob(os.path.join(training_dataset_dir, '*.jpg'))
train_mask_list = glob.glob(os.path.join(training_dataset_dir, '*mask.jpg'))

test_img_list = glob.glob(os.path.join(test_dataset_dir, '*.jpg'))
test_mask_list = glob.glob(os.path.join(test_dataset_dir, '*mask.jpg'))


## 1.1 Load images and masks in two separate arrays

In [5]:
def loadData(imgList, maskList):
    X = []
    y = []
    name_X = []
    for i in tqdm(imgList):
        for j in maskList:
            img_name = os.path.split(i)[1].split('.jpg')[0].split('_')[1]
            mask_name = os.path.split(j)[1].split('.jpg')[0].split('_')[1]

            if img_name == mask_name:            
                img = cv2.imread(i)
                mask = cv2.imread(j,0)
                X.append(img)
                y.append(mask)
                name_X.append(os.path.split(i)[1].split('.jpg')[0])
        
        
        
    return (np.array(X), np.array(y))

In [6]:
#X,y are for training and validation
X, y = loadData(train_img_list, train_mask_list)

#X_test,y_test are for testing
X_test, y_test = loadData(test_img_list, test_mask_list)

  0%|          | 0/796 [00:00<?, ?it/s]

/tmp/ipykernel_2233823/3856444674.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return (np.array(X), np.array(y))


  0%|          | 0/204 [00:00<?, ?it/s]

## 1.2 Shuffle dataset

In [7]:
#mask and image must be shuffled with the same index
def custom_shuffle(img_array, mask_array):
    assert len(img_array) == len(mask_array)
    p = np.random.permutation(len(img_array))
    return img_array[p], mask_array[p]

In [8]:
X,y=custom_shuffle(X,y)
X_test,y_test=custom_shuffle(X_test,y_test)

# 2.0 Visualize the dataset

In [9]:
img = X[1]
mask = y[1]

In [10]:
# show the mask over the image
plt.figure(figsize=(14,10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.imshow(mask, alpha=0.4)
plt.show()

# 3.0 Pre-process the dataset

In [11]:
#input shape network
in_net_h=224
in_net_w=224
net_channels=3

In [12]:
#Resize the prediction mantaining the aspect ratio for X

def resizeImage(X, width, force_dim=False, height = None):
    X_res = []
    for img in tqdm(X):
        r = width / img.shape[1]
        dim = (width, int(img.shape[0] * r))
        # not mantain the ratio
        if force_dim:
            img_resized = cv2.resize(img.astype('uint8'), (width,height), interpolation = cv2.INTER_AREA)
        else:
            img_resized = cv2.resize(img.astype('uint8'), dim, interpolation = cv2.INTER_AREA)
        X_res.append(img_resized)
        
    return np.array(X_res)

In [13]:
#Resize the prediction mantaining the aspect ratio for y
def resizeImage_y(y, width, force_dim=False, height = None):
    y_res = []
    for img,img1 in tqdm(zip(y[:,0],y[:,1])):
        
        
        r = width / img.shape[1]
        dim = (width, int(img.shape[0] * r))
        # not mantain the ratio
        if force_dim:
            img_resized = cv2.resize(img.astype('uint8'), (width,height), interpolation = cv2.INTER_AREA)
        else:
            img_resized = cv2.resize(img.astype('uint8'), dim, interpolation = cv2.INTER_AREA)
            
        r = width / img1.shape[1]
        dim = (width, int(img1.shape[0] * r))
        # not mantain the ratio
        if force_dim:
            img_resized1 = cv2.resize(img1.astype('uint8'), (width,height), interpolation = cv2.INTER_AREA)
        else:
            img_resized1 = cv2.resize(img1.astype('uint8'), dim, interpolation = cv2.INTER_AREA)
            
            
        y_res.append([img_resized,img_resized1])
    
    return np.array(y_res)

In [14]:
def normalize(X):
    return (X / 255)

## 3.1 Resize the dataset

In [15]:
#Resize the images and masks with the accepted dimensions by the network

X = resizeImage(X, in_net_w, force_dim=True, height = in_net_h)
X_test = resizeImage(X=X_test,width= in_net_w, force_dim=True, height = in_net_h)

y = resizeImage(y, in_net_w, force_dim=True, height = in_net_h)
y_test = resizeImage(X=y_test, width=in_net_w, force_dim=True, height = in_net_h)

  0%|          | 0/796 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/796 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

## 3.2 Normalize the dataset

In [16]:
X = normalize(X)
X_test = normalize(X=X_test)

In [17]:
y= normalize(y)
y_test= normalize(X=y_test)

In [18]:
#Apply a threshold to have 0,1 despite the resizing
y[y>=(0.5)]=1.0    
y[y<(0.5)]=0.0

#Apply a threshold to have 0,1 despite the resizing
y_test[y_test>=(0.5)]=1.0
y_test[y_test<(0.5)]=0.0

# 4.0 Build and Train model

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from datetime import datetime

from tensorflow.keras.models import Model,load_model

from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Activation, Input, Add, AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape,Dropout, Multiply, Flatten,UpSampling2D
from tensorflow.keras.losses import BinaryCrossentropy


from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow.keras.utils import get_custom_objects

In [20]:
#import network
from tensorflow.keras.applications import MobileNetV3Large

In [21]:
#Save checkpoints

model_dir = os.path.join(PATH_DIR, 'bin')

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
name = 'MobileNet_V3'
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "logs"
logdir = "{}/run-{}_{}/".format(root_logdir, now,name)

backup_model_path = os.path.join(model_dir, '{}.h5'.format(name))
backup_weights_path = os.path.join(model_dir, '{}_weights.h5'.format(name))


checkpointer = ModelCheckpoint(filepath=backup_weights_path, 
                               monitor = 'loss',
                               verbose=1, 
                               save_best_only=True)

## 4.1 Building model

In [22]:
#Define custom activation function
def hard_swish(x):
    return x * tf.nn.relu6(x + 3) / 6

In [35]:
#Defining dropout_r
dropout_r=0.2

#Base model mobile net
model_base= MobileNetV3Large(input_shape=(in_net_w,in_net_h,3),
                     alpha=1.0,
                     minimalistic=False,
                     include_top=False,
                     weights='imagenet',
                     input_tensor=None,
                     classes=1,
                     pooling='avg',
                     dropout_rate=dropout_r,
#                      backend=keras.backend,
#                      layers=keras.layers,
#                      models=tf.keras.models,
#                      utils=tf.keras.utils
                     )
# backend=keras.backend
# layers=keras.layers
# models=tf.keras.models
# utils=tf.keras.utils
# out_1_8= layers.get_layer('activation_15').output
# layer_names=[layer.name for layer in model_base.layers]
# print(layer_names)
# model_base.get_layer('mobilenetv3large').summary()
model_base.summary()

Model: "MobilenetV3large"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 224, 224, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 112, 112, 16) 432         rescaling_4[0][0]                
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 112, 112, 16) 64          Conv[0][0]                       
___________________________________________________________________________________

In [38]:
activation_number=41

#Adding segmentation head

def buildModel(base_model, dropout_rate= dropout_r, n_class=1): 
    global activation_number
    #1/8 resolution output
    
    out_1_8= base_model.get_layer(index=15).output #'activation_15'
    
    #1/16 resolution output
    
    out_1_16= base_model.get_layer(index=29).output #'activation_29'
    
    
    # branch1
    x1 = Conv2D(128, (1, 1))(out_1_16)
    x1 = BatchNormalization()(x1)
    
    layer_name_act="activation_head"+str(activation_number)
    x1 = Activation('relu',name=layer_name_act)(x1)
    activation_number+=1
    
    # branch2
    s = x1.shape

    #custom average pooling2D
    x2 = AveragePooling2D(pool_size=(12, 12), strides=(4, 5),data_format='channels_last')(out_1_16)
    x2 = Conv2D(128, (1, 1))(x2)
    
    
    layer_name_act="activation_head"+str(activation_number)
    
    x2 = Activation('sigmoid',name=layer_name_act)(x2)
    activation_number+=1
    

    x2 = UpSampling2D(size=(int(s[1]), int(s[2])),data_format='channels_last',interpolation="bilinear")(x2)

    
    
    # branch3
    x3 = Conv2D(n_class, (1, 1))(out_1_8)
    
    # multiply
    m1 = Multiply()([x1, x2])

    m1 = UpSampling2D(size=(2, 2),data_format='channels_last',interpolation="bilinear")(m1)
    m1 = Conv2D(n_class, (1, 1))(m1)

    # add
    m2 = Add()([m1, x3])

    
    #adding this UPsampling of factor 8
    m2 = UpSampling2D(size=(8, 8),data_format='channels_last',interpolation="bilinear")(m2)

    
    # predictions 
    layer_name_act="activation_head"+str(activation_number)
    predictions = Activation('sigmoid',name=layer_name_act)(m2)
    
    
    activation_number+=1 


    # final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [39]:
model=buildModel(base_model=model_base)

ValueError: Operands could not be broadcast together with shapes (56, 56, 128) (672, 504, 128)

In [ ]:
model.summary()

## 4.2 Training model

In [ ]:
#Define the optimizers
optimizer_r = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer_a = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
optimizer_SDG=SGD(learning_rate=0.01, momentum=0.0, nesterov=False)


In [ ]:
def loss_IoU(y_true, y_pred): 
    
    intersection_tensor=tf.math.multiply(y_true,y_pred)
    inter=tf.reduce_sum(intersection_tensor)
    
    union=tf.reduce_sum(tf.math.subtract(tf.math.add(y_true,y_pred),intersection_tensor))
    
    
    iou= tf.math.divide(inter,union)
    return 1-iou

In [ ]:
#metric
def class_IoU(y_true, y_pred):

    
    threshold = tf.constant([0.9])
    y_pred_threshold=tf.cast(tf.math.greater(y_pred, threshold),tf.int32)
    y_true=tf.cast(y_true,tf.int32)
    

    intersection_tensor=tf.math.multiply(y_true,y_pred_threshold)
    inter=tf.reduce_sum(intersection_tensor)
    
    #union= a+b-intersection
    union=tf.reduce_sum(tf.math.subtract(tf.math.add(y_true,y_pred_threshold),intersection_tensor))
    
    
    return tf.math.divide(inter,union)

### 4.2.1 Transfer learning: freeze the lower layers


Freezing the lower layers for a few epochs and then train the whole model


In [ ]:
#Compile the model 
model.compile(optimizer=optimizer_r, loss=loss_IoU, metrics = [class_IoU])

In [ ]:
#The first layers have been already frozen previously

early_stopping_ = tf.keras.callbacks.EarlyStopping(patience=3)

n_epochs = 10

# Train the model on the new data for a few epochs
history_F = model.fit(x = X, y = y,
                    batch_size = 32,
                    epochs = n_epochs,
                    validation_split = 0.1, 
                    shuffle = True,
                    callbacks = [early_stopping_]
                     )  

# 5.0 Test the Model


Test the model in different vineyard parcels

In [ ]:
model.evaluate(X_test,y_test)

# 6.0 Compute metrics for paper

## 6.1 Pixel accuracy


In [ ]:
acc_list=[]
for jj in tqdm(range(len(X_test))):
    y_pred = model.predict(X_test[jj][None,...])
    y_pred = (y_pred[0,:,:,0] > 0.9)
    temp=np.equal(y_pred,y_test[jj])
    score=np.sum(temp)
    accuracy=score/(224**2)
    acc_list.append(accuracy)

acc_nump=np.array(acc_list)
np.mean(acc_nump)

## 6.2 Precision @ different thresholds


In [ ]:
iou_list=[]
for jj in tqdm(range(len(X_test))):
    y_pred = model.predict(X_test[jj][None,...])
    iou=class_IoU(y_test[jj], y_pred[0,:,:,0])
    iou_list.append(iou.numpy())

In [ ]:
iou_list=np.array(iou_list)
print("iou greater 0.4 and %")
a1=len(np.where(iou_list>0.4)[0])
print(a1,a1/500)
print("iou greater 0.5 and %")
a1=len(np.where(iou_list>0.5)[0])
print(a1,a1/500)
print("iou greater 0.6 and %")
a1=len(np.where(iou_list>0.6)[0])
print(a1,a1/500)
print("iou greater 0.7 and %")
a1=len(np.where(iou_list>0.7)[0])
print(a1,a1/500)

# 7.0 save model

In [ ]:
#Saving the model
cwd = os.getcwd()
model_path = os.path.join(cwd,'bin','mobileNetv3_segmentation_new1.h5')
model.save(model_path, save_format='h5')

# 8.0 Test on a random image

In [ ]:
cwd = os.getcwd()
print(cwd)
model_path = os.path.join(cwd,'bin','mobileNetv3_segmentation_new1.h5')
model=tf.keras.models.load_model(model_path,
                                 custom_objects={"hard_swish": hard_swish,"loss_IoU": loss_IoU,"class_IoU":class_IoU}
                                )

In [ ]:
x_test_new = cv2.imread('frame_5260.jpg') 

x_test_new = cv2.cvtColor(x_test_new, cv2.COLOR_BGR2RGB)
x_test_new = cv2.resize(x_test_new.astype('uint8')
                        , (224,224), interpolation = cv2.INTER_AREA)
x_test_new = normalize(x_test_new)

In [ ]:
def plotPrediction(x, y_pred):
    fig, axes = plt.subplots(1, 2, figsize=(15,15))
    axes[0].imshow(x)
    axes[1].imshow(y_pred)

In [ ]:
y_pred = model.predict(x_test_new[None,...])
y_pred = (y_pred > 0.9)
plotPrediction(x_test_new, y_pred[0,:,:,0])

# 8.0 Vegetation index

In [ ]:
#Datasets_segmentation/training_dataset
img_test = cv2.imread('vegetation_index/frame_5260.jpg')
img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
img_test = cv2.resize(img_test.astype('uint8'), (224,224), interpolation = cv2.INTER_AREA)
img_test = normalize(img_test)


In [ ]:
y_pred = model.predict(x_test_new[None,...])
y_pred = (y_pred > 0.9)
plotPrediction(x_test_new, y_pred[0,:,:,0])

In [ ]:
img = cv2.imread('vegetation_index/frame_11115.jpg')

x_test_new = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
x_test_new = cv2.resize(x_test_new.astype('uint8'), (224,224), interpolation = cv2.INTER_AREA)
x_test_new = normalize(x_test_new)
img = cv2.resize(img.astype('uint8'), (224,224), interpolation = cv2.INTER_AREA)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
y_pred = model.predict(x_test_new[None,...])
y_pred = (y_pred > 0.9)

#Show the mask over the image
plt.figure(figsize=(8,8),frameon=False)
plt.axis('off')
plt.imshow(img)

#Remove pink from sky
rem_pred=y_pred[0,:,:,0].astype(np.float)

rem_pred[rem_pred==0]=np.NaN
plt.imshow(rem_pred, alpha=0.4,cmap='Spectral')

plt.savefig('s_veg2.png', bbox_inches='tight', pad_inches=0)

In [ ]:
#Reverse colours
gira=cv2.imread('s_veg2.png')
gira=cv2.cvtColor(gira, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8,8),frameon=False)
plt.axis('off')
plt.imshow(gira)
plt.savefig('s_veg2.png', bbox_inches='tight', pad_inches=0)

In [ ]:
output=y_pred[0,:,:,0]

In [ ]:
#Compute vegetation index
veg=np.zeros((224,224))

for x,y in zip(np.where(output)[0],np.where(output)[1]):
    veg[x,y]=(img[x,y,1]-img[x,y,0])/(img[x,y,1]+img[x,y,0]-img[x,y,2])


In [ ]:
#Remove outliers 
veg[veg>1]=np.NaN 
veg[veg==0]=np.NaN
veg[veg==np.inf]=np.NaN

In [6]:
from sklearn.preprocessing import MinMaxScaler

#Normalize
min_max_scaler = MinMaxScaler()
map_veg = min_max_scaler.fit_transform(veg)

In [ ]:
plt.figure(figsize=(8,8))
plt.axis('off')

cf=plt.imshow(map_veg,cmap='inferno')
cd= plt.colorbar(cf)

cd.set_ticklabels(['LOW','HIGH'])
plt.show()